# Appendix E: Comparison with Traditional Sampler

This notebook reproduces the comparison analysis from the main cogwheel comparison notebook, with hardcoded injection parameters for masses 20, 50, and 100 as specified in Table III of https://arxiv.org/abs/2507.16022.


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
import json
import matplotlib.pyplot as plt
import pandas as pd
import pstats

import numpy as np
from cogwheel import data, gw_utils, gw_plotting, utils
from cogwheel import sampling
from cogwheel import posterior
from dot_pe import inference


# Mass ~ 20 Msun Analysis


In [ ]:
# Generate event data for mass ~20
event_data_kwargs = {
    "detector_names": "HLV",
    "duration": 120.0,
    "asd_funcs": ["asd_H_O3", "asd_L_O3", "asd_V_O3"],
    "tgps": 0.0,
    "fmax": 1600.0,
}

event_data_20 = data.EventData.gaussian_noise(
    eventname="example_mchirp_20", **event_data_kwargs, seed=20250407
)

# Use hardcoded injection parameters for mass 20
injection_par_dic_20 = {
    "ra": 3.6107815139655437,
    "dec": -1.1438647962369957,
    "iota": 1.4151318259413483,
    "psi": 0.9757967663751129,
    "phi_ref": 4.074052036195291,
    "s1z": 0.6,
    "s2z": 0.6,
    "s1x_n": 0.5,
    "s1y_n": 0.5,
    "s2x_n": 0.5,
    "s2y_n": 0.5,
    "l1": 0.0,
    "l2": 0.0,
    "tgps": 0.0,
    "f_ref": 50.0,
    "d_luminosity": 2000.0,
    "t_geocenter": 0.0,
    "m1": 33.90281400593718,
    "m2": 22.601876003958147,
}


event_data_20.inject_signal(injection_par_dic_20, "IMRPhenomXODE")

In [ ]:
# Run dot-PE analysis for mass ~20
# Bank path from create_banks.py - banks are created in current working directory
bank_folder_20 = Path("bank_mchirp_20")

print(f"Using bank folder: {bank_folder_20.absolute()}")
print(f"Bank exists: {bank_folder_20.exists()}")

rundir_20 = inference.run_and_profile(
    event_dir=Path("example_mchirp_20"),
    event=event_data_20,
    bank_folder=bank_folder_20,
    n_int=2**16,
    n_ext=2048,
    n_phi=32,
    n_t=64,
    i_int_start=0,
    blocksize=2**10,
    single_detector_blocksize=2**10,
    seed=20250703,
    size_limit=10**7,
    draw_subset=True,
    n_draws=None,
)


In [ ]:
# Load and analyze results
dot_pe_rundirs = sorted(Path("example_mchirp_20").glob("run_*"))

if dot_pe_rundirs:
    dot_pe_rundir = dot_pe_rundirs[-1]  # Get the latest run

    # Load results
    dot_pe_samples = pd.read_feather(dot_pe_rundir / "samples.feather")

    # Load runtime info
    dot_pe_profile = pstats.Stats(str(dot_pe_rundir / "profile_output.prof"))
    dot_pe_runtime = dot_pe_profile.total_tt

    # Load summary results
    dot_pe_summary_results = utils.read_json(dot_pe_rundir / "summary_results.json")

In [ ]:
bank_config = utils.read_json(Path(bank_folder_20) / "bank_config.json")
mchirp_range = (bank_config["min_mchirp"], bank_config["max_mchirp"])
q_min = bank_config["q_min"]

post = posterior.Posterior.from_event(
    event_data_20,
    mchirp_guess=None,
    approximant="IMRPhenomXODE",
    prior_class="CartesianIntrinsicIASPrior",
    prior_kwargs={
        "mchirp_range": mchirp_range,
        "q_min": q_min,
        "f_ref": 50.0,
    },
    ref_wf_finder_kwargs={"f_ref": 50.0},
)
sampler = sampling.Nautilus(post, run_kwargs=dict(n_live=1000))
event_dir = Path("./cogwheel_runs") / event_data_20.eventname
if not event_dir.exists():
    utils.mkdirs(event_dir)

cogwheel_rundir = sampler.get_rundir(event_dir)
sampler.run(cogwheel_rundir)

In [ ]:
cogwheel_samples = pd.read_feather(Path(cogwheel_rundir) / "samples.feather")

In [ ]:
dot_pe_label = r"$\mathtt{dot-PE}$ (This work)"
cogwheel_label = r"Stochastic Sampler $(\mathtt{cogwheel})$"

mcp = gw_plotting.MultiCornerPlot(
    [dot_pe_samples, cogwheel_samples],
    labels=[dot_pe_label, cogwheel_label],
    params=[
        "mchirp",
        "lnq",
        "chieff",
        "iota",
        "psi",
        "ra",
        "dec",
        "d_luminosity",
        "t_geocenter",
        "phi_ref",
        "lnl",
    ],
    smooth=1,
    tail_probability=1e-2,
)
mcp.plot()
mcp.corner_plots[0].axes[0][-1].legend(
    *mcp.corner_plots[0].axes[0][0].get_legend_handles_labels(),
    fontsize=14,
)
plt.tight_layout()

plt.savefig("figure-7.pdf")


# Mass ~ 50 Msun Analysis


In [ ]:
# Generate event data for mass ~50 using exact parameters from example_mchirp_50.json
event_data_50 = data.EventData.gaussian_noise(
    eventname="example_mchirp_50", **event_data_kwargs, seed=20250406
)

# Use exact injection parameters from example_mchirp_50.json
injection_par_dic_50 = {
    "ra": 5.512004870562566,
    "dec": -0.48153226859677867,
    "iota": 2.3050925075889075,
    "psi": 0.5861847706555092,
    "phi_ref": 2.9838688587424085,
    "s1z": 0.6,
    "s2z": 0.6,
    "s1x_n": 0.5,
    "s1y_n": -0.5,
    "s2x_n": -0.5,
    "s2y_n": 0.5,
    "l1": 0.0,
    "l2": 0.0,
    "tgps": 0.0,
    "f_ref": 50.0,
    "d_luminosity": 3000.0,
    "t_geocenter": 0.0,
    "m1": 115.06262806620583,
    "m2": 57.53131403310289
}

event_data_50.inject_signal(injection_par_dic_50, "IMRPhenomXODE")


In [ ]:
# Run dot-PE analysis for mass ~50
bank_folder_50 = Path("bank_mchirp_50")

rundir_50 = inference.run_and_profile(
    event_dir=Path("example_mchirp_50"),
    event=event_data_50,
    bank_folder=bank_folder_50,
    n_int=2**16,
    n_ext=2048,
    n_phi=32,
    n_t=64,
    i_int_start=0,
    blocksize=2**10,
    single_detector_blocksize=2**10,
    seed=20250703,
    size_limit=10**7,
    draw_subset=True,
    n_draws=None,
    use_reps=False,
)


In [ ]:
# Load dot-PE results for mass 50
dot_pe_rundirs_50 = sorted(Path("example_mchirp_50").glob("run_*"))
if dot_pe_rundirs_50:
    dot_pe_samples_50 = pd.read_feather(dot_pe_rundirs_50[-1] / "samples.feather")

# Run cogwheel for mass 50
bank_config_50 = utils.read_json(Path(bank_folder_50) / "bank_config.json")
mchirp_range_50 = (bank_config_50["min_mchirp"], bank_config_50["max_mchirp"])
q_min_50 = bank_config_50["q_min"]

post_50 = posterior.Posterior.from_event(
    event_data_50,
    mchirp_guess=None,
    approximant="IMRPhenomXODE",
    prior_class="CartesianIntrinsicIASPrior",
    prior_kwargs={
        "mchirp_range": mchirp_range_50,
        "q_min": q_min_50,
        "f_ref": 50.0,
    },
    ref_wf_finder_kwargs={"f_ref": 50.0},
)
sampler_50 = sampling.Nautilus(post_50, run_kwargs=dict(n_live=1000))
event_dir_50 = Path("./cogwheel_runs") / event_data_50.eventname
if not event_dir_50.exists():
    utils.mkdirs(event_dir_50)

cogwheel_rundir_50 = sampler_50.get_rundir(event_dir_50)
sampler_50.run(cogwheel_rundir_50)


In [ ]:
# Load cogwheel results for mass 50 and create comparison plot
cogwheel_samples_50 = pd.read_feather(Path(cogwheel_rundir_50) / "samples.feather")

dot_pe_label = r"$\mathtt{dot-PE}$ (This work)"
cogwheel_label = r"Stochastic Sampler $(\mathtt{cogwheel})$"

mcp_50 = gw_plotting.MultiCornerPlot(
    [dot_pe_samples_50, cogwheel_samples_50],
    labels=[dot_pe_label, cogwheel_label],
    params=[
        "mchirp",
        "lnq",
        "chieff",
        "iota",
        "psi",
        "ra",
        "dec",
        "d_luminosity",
        "t_geocenter",
        "phi_ref",
        "lnl",
    ],
    smooth=1,
    tail_probability=1e-2,
)
mcp_50.plot()
mcp_50.corner_plots[0].axes[0][-1].legend(
    *mcp_50.corner_plots[0].axes[0][0].get_legend_handles_labels(),
    fontsize=14,
)
plt.tight_layout()
plt.savefig("figure-8.pdf")


# Mass ~ 100 Msun Analysis


In [ ]:
# Generate event data for mass ~100 using exact parameters from example_mchirp_100.json
event_data_100 = data.EventData.gaussian_noise(
    eventname="example_mchirp_100", **event_data_kwargs, seed=20250407
)

# Use exact injection parameters from example_mchirp_100.json
injection_par_dic_100 = {
    "ra": 2.3394832559272873,
    "dec": 0.6994178496616139,
    "iota": 2.364842518017542,
    "psi": 2.297268304342178,
    "phi_ref": 2.5697179725937414,
    "s1z": 0.3,
    "s2z": 0.9,
    "s1x_n": 0.7,
    "s1y_n": 0.6,
    "s2x_n": 0.1,
    "s2y_n": 0.1,
    "l1": 0.0,
    "l2": 0.0,
    "tgps": 0.0,
    "f_ref": 50.0,
    "d_luminosity": 4000.0,
    "t_geocenter": 0.0,
    "m1": 328.7503659034453,
    "m2": 164.37518295172256
}

event_data_100.inject_signal(injection_par_dic_100, "IMRPhenomXODE")


In [ ]:
# Run dot-PE analysis for mass ~100
bank_folder_100 = Path("bank_mchirp_100")

rundir_100 = inference.run_and_profile(
    event_dir=Path("example_mchirp_100"),
    event=event_data_100,
    bank_folder=bank_folder_100,
    n_int=2**16,
    n_ext=2048,
    n_phi=32,
    n_t=64,
    i_int_start=0,
    blocksize=2**10,
    single_detector_blocksize=2**10,
    seed=20250703,
    size_limit=10**7,
    draw_subset=True,
    n_draws=None,
    use_reps=False,
)


In [ ]:
# Load dot-PE results for mass 100
dot_pe_rundirs_100 = sorted(Path("example_mchirp_100").glob("run_*"))
if dot_pe_rundirs_100:
    dot_pe_samples_100 = pd.read_feather(dot_pe_rundirs_100[-1] / "samples.feather")

# Run cogwheel for mass 100
bank_config_100 = utils.read_json(Path(bank_folder_100) / "bank_config.json")
mchirp_range_100 = (bank_config_100["min_mchirp"], bank_config_100["max_mchirp"])
q_min_100 = bank_config_100["q_min"]

post_100 = posterior.Posterior.from_event(
    event_data_100,
    mchirp_guess=None,
    approximant="IMRPhenomXODE",
    prior_class="CartesianIntrinsicIASPrior",
    prior_kwargs={
        "mchirp_range": mchirp_range_100,
        "q_min": q_min_100,
        "f_ref": 50.0,
    },
    ref_wf_finder_kwargs={"f_ref": 50.0},
)
sampler_100 = sampling.Nautilus(post_100, run_kwargs=dict(n_live=1000))
event_dir_100 = Path("./cogwheel_runs") / event_data_100.eventname
if not event_dir_100.exists():
    utils.mkdirs(event_dir_100)

cogwheel_rundir_100 = sampler_100.get_rundir(event_dir_100)
sampler_100.run(cogwheel_rundir_100)


In [ ]:
# Load cogwheel results for mass 100 and create comparison plot
cogwheel_samples_100 = pd.read_feather(Path(cogwheel_rundir_100) / "samples.feather")

mcp_100 = gw_plotting.MultiCornerPlot(
    [dot_pe_samples_100, cogwheel_samples_100],
    labels=[dot_pe_label, cogwheel_label],
    params=[
        "mchirp",
        "lnq",
        "chieff",
        "iota",
        "psi",
        "ra",
        "dec",
        "d_luminosity",
        "t_geocenter",
        "phi_ref",
        "lnl",
    ],
    smooth=1,
    tail_probability=1e-2,
)
mcp_100.plot()
mcp_100.corner_plots[0].axes[0][-1].legend(
    *mcp_100.corner_plots[0].axes[0][0].get_legend_handles_labels(),
    fontsize=14,
)
plt.tight_layout()
plt.savefig("figure-9.pdf")
